In [2]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
   
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
   
def download_pdf(url,path,filename):
    try:
       
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
   
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

   
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [ ]:
source = "https://www.courts.state.co.us/Forms/Index.cfm"

soup = connect_to_page(source)

cat_names = []
cat_refs = []

starting_soup = soup.find_all("a",href = True)
for i in starting_soup:
    if "Category" in i['href']:
        cat_refs.append(i['href'])
        cat_names.append(i.text.split("\n")[0])
print(cat_refs)
print(cat_names)

In [ ]:
today = date.today().strftime("%Y-%m-%d")
global files_df 
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
form_dest = "CO/CO_All_forms/"
jur = "CO"
pdfs = {}
subcats = []

def no_subcats(soup, files_df):
    for i in soup.find_all("span", class_= "form-box"):
        url = i.find("a",href= True)['href']
        filename = i.text.split("\n")[1].split("-")[0]
        title = i.text.split("\n")[1].split("-")[1]
        form_type = i.text.split("\n")[2].split(" ")[1]
        id = hashme(url)
        

        if filename not in pdfs:
            pdfs[filename] = title
            print(url,"-----",filename,"-------",title,"--------", form_type)

            if form_type == "PDF":
                download_pdf(url, form_dest, id + ".pdf")
            else:
                download_docx(url,id + ".doc")
            files_df = files_df.append(pd.DataFrame([[id,jur,source,title,group,url,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]))
    return files_df

for i in range(len(cat_refs)): #in range cat refs
    soup = connect_to_page(cat_refs[i])
    table = soup.find("td")
    table_cats = table.find_all("a",href = True)
    group = cat_names[i]
    if len(table_cats) > 0: 
        for ref in table_cats:
            files_df = no_subcats(connect_to_page(ref['href']),files_df)

    else:
        
        files_df = no_subcats(soup, files_df)

files_df.to_csv("CO-v1.csv",index=False, encoding="utf-8")

